# Machine Learning Trading Bot - Template

Template assembled by referring to Algo trading activities and optional homework assignment

In [2]:
# Import libraries
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from finta import TA
import nolds

# Import CSV file and create pandas DataFrame

In [3]:
# Import stocks dataset
loblaws_df = pd.read_csv(Path('./Resources/loblaw.csv'), index_col= 'Date', parse_dates = True, infer_datetime_format=True)
bmo_df = pd.read_csv(Path('./Resources/bmo.csv'), index_col= 'Date', parse_dates = True, infer_datetime_format=True)
tri_df = pd.read_csv(Path('./Resources/tri.csv'), index_col= 'Date', parse_dates = True, infer_datetime_format=True)
cix_df = pd.read_csv(Path('./Resources/cix.csv'), index_col= 'Date', parse_dates = True, infer_datetime_format=True)

# Review the DataFrame
loblaws_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,60.970001,61.799999,60.450001,61.049999,57.818451,626700
2019-01-03,60.770000,61.110001,60.369999,60.669998,57.458572,617800
2019-01-04,60.930000,61.290001,60.509998,60.799999,57.581692,444100
2019-01-07,61.000000,61.599998,60.470001,61.369999,58.121513,724700
2019-01-08,61.549999,62.090000,61.310001,61.849998,58.576099,601100


In [4]:
def get_hurst_exponent(time_series, max_lag=20):
    """Returns the Hurst Exponent of the time series"""
    
    lags = range(2, max_lag)

    # variances of the lagged differences
    tau = [np.std(np.subtract(time_series[lag:], time_series[:-lag])) for lag in lags]

    # calculate the slope of the log plot -> the Hurst Exponent
    reg = np.polyfit(np.log(lags), np.log(tau), 1)

    return reg[0]

In [7]:
for lag in [20, 100, 300, 500]:
    hurst_exp = get_hurst_exponent(loblaws_df["Adj Close"].values, lag)
    print(f"Hurst exponent with {lag} lags: {hurst_exp:.4f}")

Hurst exponent with 20 lags: 0.4770
Hurst exponent with 100 lags: 0.4746
Hurst exponent with 300 lags: 0.5463
Hurst exponent with 500 lags: 0.4366


In [8]:
for lag in [20, 100, 300, 500]:
    hurst_exp = get_hurst_exponent(bmo_df["Adj Close"].values, lag)
    print(f"Hurst exponent with {lag} lags: {hurst_exp:.4f}")

Hurst exponent with 20 lags: 0.5727
Hurst exponent with 100 lags: 0.5559
Hurst exponent with 300 lags: 0.6054
Hurst exponent with 500 lags: 0.4701


In [9]:
for lag in [20, 100, 300, 500]:
    hurst_exp = get_hurst_exponent(tri_df["Adj Close"].values, lag)
    print(f"Hurst exponent with {lag} lags: {hurst_exp:.4f}")

Hurst exponent with 20 lags: 0.5574
Hurst exponent with 100 lags: 0.4499
Hurst exponent with 300 lags: 0.3384
Hurst exponent with 500 lags: 0.2298


In [10]:
for lag in [20, 100, 300, 500]:
    hurst_exp = get_hurst_exponent(cix_df["Adj Close"].values, lag)
    print(f"Hurst exponent with {lag} lags: {hurst_exp:.4f}")

Hurst exponent with 20 lags: 0.6234
Hurst exponent with 100 lags: 0.5202
Hurst exponent with 300 lags: 0.4100
Hurst exponent with 500 lags: 0.2774


In [ ]:
#Filter the date index and close columns
signals_df = stocks_df#.loc[:, ['close']]

# Use pct_change function to generate returns from close prices
signals_df['Actual Returns'] = signals_df['Close'].pct_change()

#Drop all NaN values from the DataFrame
signals_df = signals_df.dropna()

#Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

# Create a trading algorithm using technical indicators

## Generate Signals - Example using SMA with short and long window

In [ ]:
# Create SMA signals DataFrame
#signals_df = stocks_df.loc[:, ["close"]].copy()

# Set the short and long windows
short_window = 50
long_window = 100

# Create a short window SMA and long window SMA
signals_df['SMA_Fast'] = signals_df['Close'].rolling(window=short_window).mean()
signals_df['SMA_Slow'] = signals_df['Close'].rolling(window=long_window).mean()

signals_df = signals_df.dropna()

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

In [ ]:
# Initialize the new Signal column
signals_df['SMA Signal'] = 0.0

# When Actual Returns are greater than or equal to 0, generate signal to buy stock long
signals_df.loc[(signals_df['Actual Returns'] >= 0), 'Signal'] = 1.0

# When Actual Returns are less than 0, generate signal to sell stock short
signals_df.loc[(signals_df['Actual Returns'] < 0), 'Signal'] = 0.0

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

In [ ]:
# Review value counts
signals_df['Signal'].value_counts()

In [ ]:
# Visualize close price for the investment
stocks_close = signals_df[["Close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[["SMA_Fast", "SMA_Slow"]].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
sma_ema_plot = stocks_close * moving_avgs
sma_ema_plot

In [ ]:
# Calculate the strategy returns and add them to the signals_df DataFrame
signals_df['Strategy Returns'] = signals_df['Actual Returns'] * signals_df['Signal'].shift()

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

In [ ]:
# Plot Strategy Returns to examine performance
(1 + signals_df['Strategy Returns']).cumprod().hvplot(title='Strategy Returns')

## Example using EMA technical indicators

In [ ]:
# Create EMA signals DataFrame
# signals_df = stocks_df.loc[:, ["close"]].copy()

# Set the short window and long windows
short_window = 50
long_window = 100

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df["EMA_Short"] = signals_df["Close"].ewm(span=short_window).mean()
signals_df["EMA_Long"] = signals_df["Close"].ewm(span=long_window).mean()

# Review the DataFrame
signals_df.head()

In [ ]:
# Set the signal column
signals_df['Signal'] = 0

# Generate the trading signal 1 or 0
# Where 1 is when the EMA50 is greater than (i.e. crosses over) the EMA100
# Where 0 is when the EMA 50 is under the EMA100
signals_df['Signal'][short_window:] = np.where(
    signals_df['EMA_Short'][short_window:] > signals_df['EMA_Long'][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or 0
signals_df['EMA Entry/Exit'] = signals_df['Signal'].diff()

# Review DataFrame
signals_df.head()

In [ ]:
# Review value counts
signals_df['EMA Entry/Exit'].value_counts()

In [ ]:
# Visualize entry position relative to close price
entry = signals_df[signals_df["EMA Entry/Exit"] == 1.0]["Close"].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = signals_df[signals_df["EMA Entry/Exit"] == 0.0]["Close"].hvplot.scatter(
    color='orange',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[["Close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[["EMA_Short", "EMA_Long"]].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot


In [ ]:
# Calculate the strategy returns and add them to the signals_df DataFrame
signals_df['Strategy Returns'] = signals_df['Actual Returns'] * signals_df['Signal'].shift()

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

In [ ]:
# Plot Strategy Returns to examine performance
(1 + signals_df['Strategy Returns']).cumprod().hvplot(title='Strategy Returns')

# Example - Using Bollinger Bands

In [ ]:
# Create Bollinger Bands signals DataFrame
bb_stocks_df = TA.BBANDS(stocks_df)

# Review DataFrame
bb_stocks_df.tail()

In [ ]:
# Concatenate the Bollinger Bands to the stocks DataFrame
bb_signals_df = pd.concat([stocks_df, bb_stocks_df])

# Review the DataFrame
bb_signals_df.tail()

In [ ]:
# Visualize close price for stocks
security_close = bb_signals_df[["Close"]].hvplot(
    line_color="lightgray",
    ylabel="Price in $",
    width=1000,
    height=400
)

bb_upper = bb_signals_df[["BB_UPPER"]].hvplot(
    line_color="purple",
    ylabel="Price in $",
    width=1000,
    height=400
)


bb_middle = bb_signals_df[["BB_MIDDLE"]].hvplot(
    line_color="orange",
    ylabel="Price in $",
    width=1000,
    height=400
)

bb_lower = bb_signals_df[["BB_LOWER"]].hvplot(
    line_color="blue",
    ylabel="Price in $",
    width=1000,
    height=400
)


# Overlay plots
bbands_plot = security_close * bb_upper * bb_middle * bb_lower
bbands_plot

# Create Trading Algorithm Using Bollinger Bands

In [ ]:
# Create a Trading Algorithm Using Bollinger Bands
# Set the Signal column
bb_signals_df['Signal'] = 0.0

# Generate the trading signals 1 (entry) or -1 (exit) for a long position trading algorithm
# where 1 is when the Close price is less than the BB_LOWER window
# where -1 is when the Close price is greater the the BB_UPPER window
for index, row in bb_stocks_df.iterrows():
    if row["Close"] < row["BB_LOWER"]:
        bb_stocks_df.loc[index, "Signal"] = 1.0
    if row["Close"] > row["BB_UPPER"]:
        bb_stocks_df.loc[index,"Signal"] = 0.0

# Review the DataFrame
bb_stocks_df.tail()

# Visualize Entry/Exit Positions using Bollinger Bands

In [ ]:
# Visualize entry position relative to close price
entry = bb_stocks_df[bb_signals_df["Signal"] == 1.0]["close"].hvplot.scatter(
    color="green",
    marker="^",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = bb_stocks_df[bb_signals_df["Signal"] == 0.0]["close"].hvplot.scatter(
    color="red",
    marker="v",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = bb_signals_df[["close"]].hvplot(
    line_color="lightgray",
    ylabel="Price in $",
    width=1000,
    height=400
)

bb_upper = bb_signals_df[["BB_UPPER"]].hvplot(
    line_color="purple",
    ylabel="Price in $",
    width=1000,
    height=400
)


bb_middle = bb_signals_df[["BB_MIDDLE"]].hvplot(
    line_color="orange",
    ylabel="Price in $",
    width=1000,
    height=400
)

bb_lower = bb_signals_df[["BB_LOWER"]].hvplot(
    line_color="blue",
    ylabel="Price in $",
    width=1000,
    height=400
)


# Overlay plots
bbands_plot = security_close * bb_upper * bb_middle * bb_lower * entry * exit
bbands_plot

In [ ]:
# Calculate the strategy returns and add them to the signals_df DataFrame
bb_signals_df['Strategy Returns'] = bb_signals_df['Actual Returns'] * bb_signals_df['Signal'].shift()

# Review the DataFrame
display(bb_signals_df.head())
display(bb_signals_df.tail())

In [ ]:
# Plot Strategy Returns to examine performance
(1 + bb_signals_df['Strategy Returns']).cumprod().hvplot(title='Strategy Returns'

# Split Data into Training and Testing datasets

## SMA Example

In [ ]:
# Assign a copy of the sma_fast and sma_slow columns to a features DataFrame called X
X = signals_df[['SMA_Fast', 'SMA_Slow']].shift().dropna().copy()

# Review the DataFrame
X.head()

In [ ]:
# Create the target set selecting the Signal column and assiging it to y
y = signals_df['Signal'].copy()

# Review the value counts
y.value_counts()

In [ ]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

In [ ]:
# Select the ending period for the training data with an offset of 48 months
training_end = training_begin + DateOffset(months=48)
# Display the training end date
print(training_end)

In [ ]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
X_train.head()

In [ ]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

# Review the X_test DataFrame
X_test.head()

In [ ]:
# Scale the features DataFrames

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# From instantiate classifier model instance
model = svm.SVC()
 
# Fit the model to the data using the training data
model = model.fit(X_train_scaled, y_train)
 
# Use the testing data to make the model predictions
predictions = model.predict(X_test_scaled)

# Review the model's predicted values
predictions[:10]

In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
testing_report = classification_report(y_test, predictions)

# Print the classification report
print(testing_report)

In [ ]:
# Create a new empty predictions DataFrame:

# Create a predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index)

# Add the SVM model predictions to the DataFrame
predictions_df['Predicted'] = svm_pred

# Add the actual returns to the DataFrame
predictions_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
predictions_df['Strategy Returns'] = signals_df['Strategy Returns']

# Review the DataFrame
display(predictions_df.head())
display(predictions_df.tail())

In [ ]:
# Plot the actual returns versus the strategy returns
(1 + predictions_df[['Actual Returns', 'Strategy Returns']]).cumprod().hvplot(
    title='[ML Model] Strategy Returns'
    )

# Tune the Baseline Trading Algorithm

## Make Adjustments with parameters or with different model

### Tuning options include tuning windows of SMA/EMA, adjusting train/test split, include/exclude indicators/signals

# Evaluate a New Machine Learning Classifier and compare models

In [ ]:
# Import a new classifier from SKLearn
from sklearn.ensemble import RandomForestClassifier

# Initiate the model instance
ml_model = RandomForestClassifier()

In [ ]:
# Using the original training data as the baseline model, fit another model with the new classifier.
# Fit the model using the training data
ml_model.fit(X_train_scaled, y_train)

# Use the testing dataset to generate the predictions for the new model
ml_predictions = ml_model.predict(X_test_scaled)

# Review the model's predicted values
ml_predictions[:10]


In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
ml_testing_report = classification_report(y_test, ml_predictions)

# Print the classification report
print(ml_testing_report)

In [ ]:
# Create a new empty predictions DataFrame:

# Create a predictions DataFrame
ml_predictions_df = pd.DataFrame(index=X_test.index)

# Add the Logistic Regression model predictions to the DataFrame
ml_predictions_df['Strategy Returns'] = ml_pred

# Add the actual returns to the DataFrame
ml_predictions_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
ml_predictions_df['Trading Algorithm Returns'] = (
    ml_predictions_df['Actual Returns'] * ml_predictions_df['Strategy Returns']
)

# Review the DataFrame
display(ml_predictions_df.head())
display(ml_predictions_df.tail())

In [ ]:
# Plot the actual returns versus the strategy returns
(1 + ml_predictions_df[['Actual Returns', 'Trading Algorithm Returns']]).cumprod().hvplot(
    title='[ML Model] Strategy Returns'
    )

# Backtesting

In [ ]:
# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = 500

In [ ]:
# Buy a 500 share position when the dual moving average crossover Signal equals 1
# Otherwise, `Position` should be zero (sell)
signals_df['Position'] = share_size * signals_df['Signal']

In [ ]:
# Determine the points in time where a 500 share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

In [ ]:
# Multiply the close price by the number of shares held, or the Position
signals_df['Portfolio Holdings'] = signals_df['close'] * signals_df['Position']

In [ ]:
# Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
signals_df['Portfolio Cash'] = initial_capital - (signals_df['close'] * signals_df['Entry/Exit Position']).cumsum()

In [ ]:
# Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

In [ ]:
# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()

In [ ]:
# Print the DataFrame
signals_df.head(150)

In [ ]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='yellow',
    marker='v',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='purple',
    marker='^',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize the value of the total portfolio
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    xlabel='Date',
    width=1000,
    height=400
)

# Overlay the plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title="Apple Algorithm - Total Portfolio Value",
    yformatter='%.0f'
)

# Assessing Risk/Reward

# Calculate the Metrics

## Setup the Portfolio-Level Risk/Reward Evaluation Metrics

In [ ]:
# Create a list for the column name
columns = ["Backtest"]

# Create a list holding the names of the new evaluation metrics
metrics = [
    "Annualized Return",
    "Cumulative Returns",
    "Annual Volatility",
    "Sharpe Ratio",
    "Sortino Ratio"]

# Initialize the DataFrame with index set to the evaluation metrics and the column
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)

# Review the DataFrame
portfolio_evaluation_df

# Calculate Metrics

## Annualized Returns

In [ ]:
# Calculate annualized return
portfolio_evaluation_df.loc["Annualized Return"] = (
    signals_df["Portfolio Daily Returns"].mean() * 252
)

# Review the result
portfolio_evaluation_df

## Cumulative Returns

In [ ]:
# Calculate cumulative return
portfolio_evaluation_df.loc["Cumulative Returns"] = signals_df["Portfolio Cumulative Returns"][-1]

# Review the result
portfolio_evaluation_df

## Annual Volatility

In [ ]:
### Annual Volatility
# Calculate annual volatility
portfolio_evaluation_df.loc["Annual Volatility"] = (
    signals_df["Portfolio Daily Returns"].std() * np.sqrt(252)
)

# Review the result
portfolio_evaluation_df

## Sharpe Ratio

In [ ]:
# Calculate Sharpe ratio
portfolio_evaluation_df.loc["Sharpe Ratio"] = (
    signals_df["Portfolio Daily Returns"].mean() * 252) / (
    signals_df["Portfolio Daily Returns"].std() * np.sqrt(252)
)

# Review the result
portfolio_evaluation_df

# Sortino Ratio

### Calculate downside returns

In [ ]:
# Create a DataFrame that contains the Portfolio Daily Returns column
sortino_ratio_df = signals_df[["Portfolio Daily Returns"]].copy()

# Create a column to hold downside return values
sortino_ratio_df.loc[:,"Downside Returns"] = 0

# Find Portfolio Daily Returns values less than 0,
# square those values, and add them to the Downside Returns column
sortino_ratio_df.loc[sortino_ratio_df["Portfolio Daily Returns"] < 0,
                     "Downside Returns"] = sortino_ratio_df["Portfolio Daily Returns"]**2

# Review the DataFrame
sortino_ratio_df.tail()

In [ ]:
# Calculate the annualized downside standard deviation value
downside_standard_deviation = (
    np.sqrt(sortino_ratio_df["Downside Returns"].std()) * np.sqrt(252)
)

# Print the result
print(f"Annualized Downside STD: {downside_standard_deviation}")

In [ ]:
# The Sortino ratio is reached by dividing the annualized return value
# by the downside standard deviation value
sortino_ratio = annualized_return/downside_standard_deviation

# Add the Sortino ratio to the evaluation DataFrame
portfolio_evaluation_df.loc["Sortino Ratio"] = sortino_ratio

# Review the DataFrame
portfolio_evaluation_df

In [ ]:
# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row["Entry/Exit"] == 1:
        entry_date = index
        entry_portfolio_holding = row["Portfolio Holdings"]
        share_size = row["Entry/Exit Position"]
        entry_share_price = row["close"]

    elif row["Entry/Exit"] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row["close"] * row["Entry/Exit Position"])
        exit_share_price = row["close"]
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                "Stock": "AAPL",
                "Entry Date": entry_date,
                "Exit Date": exit_date,
                "Shares": share_size,
                "Entry Share Price": entry_share_price,
                "Exit Share Price": exit_share_price,
                "Entry Portfolio Holding": entry_portfolio_holding,
                "Exit Portfolio Holding": exit_portfolio_holding,
                "Profit/Loss": profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

# Assessing Results

In [ ]:
# Initialize share size and accumulated shares
share_size = 100
accumulated_shares = 0

# Initialize variable to hold previous day's trading price
# Set the initial value of the previous_price to 0
previous_price = 0

In [ ]:
# Loop through the Pandas DataFrame and initiate a trade at each iteration
for index, row in df.iterrows():

    # buy if the previous_price is 0, in other words, buy on the first day
    if previous_price == 0:
        df.loc[index, "trade_type"] = "buy"

        # calculate the cost of the trade by multiplying the current day's price
        # by the share_size, or number of shares purchased
        df.loc[index, "cost/proceeds"] = -(row["close"] * share_size)

        # add the number of shares purchased to the accumulated shares
        accumulated_shares += share_size

    # buy if the current day's price is less than the previous day's price
    elif row["close"] < previous_price:
        df.loc[index, "trade_type"] = "buy"

        # calculate the cost of the trade by multiplying the current day's price
        # by the share_size, or number of shares purchased
        df.loc[index, "cost/proceeds"] = -(row["close"] * share_size)

        # add the number of shares purchased to the accumulated shares
        accumulated_shares += share_size

    # hold if the current day's price is greater than the previous day's price
    elif row["close"] > previous_price:
        df.loc[index, "trade_type"] = "hold"

    # hold if the current day's price is equal to the previous day's price
    else:
        df.loc[index, "trade_type"] = "hold"

    # update the previous_price to the current row's price
    previous_price = row["close"]

    # if the index is the last index of the DataFrame, sell
    if index == df.index[-1]:
        df.loc[index, "trade_type"] = "sell"

        # calculate the proceeds by multiplying the last day's price by the accumulated shares
        df.loc[index, "cost/proceeds"] = row["close"] * accumulated_shares

In [ ]:
# Review the DataFrame
df.head

## Calculate Total Profit or Loss

In [ ]:
# Calculate the total profit/loss for 100 share size orders
total_profit_loss = round(df["cost/proceeds"].sum(), 2)

# Print the profit/loss metrics
print(f"The total profit/loss of the trading strategy is ${total_profit_loss}.")

## Calculate the Return on Investment

In [ ]:
# Initialize the variable to hold the value of the invested capital
invested_capital = 0

# Calculate the invested capital by adding the cost of all buy trades
for index, row in df.iterrows():
    if row["trade_type"] == "buy":
        invested_capital = invested_capital + row["cost/proceeds"]


# Calculate the return on investment (ROI)
roi = round((total_profit_loss / -(invested_capital)) * 100, 2)

# Print the ROI
print(f"The trading algorithm resulted in a return on investment of {roi}%")